In [111]:
import pandas as pd
import numpy as np
import re 
#import geopandas as gpd
import pandas as pd
import json
import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_theme(style="whitegrid")
import warnings

warnings.filterwarnings("ignore")

In [112]:
budgets_fbi = pd.read_csv('CleanData/Merged_FBI_Budgets_Normalized.csv')
budgets_fbi.head()

,city_merge_name,population,city_population,year,violent_crime,property_crime,rev_total,police,education_services,education,educ_higher,educ_elem_sec,social_services,public_welfare,welfare_cash,welfare_vendors,welfare_other,correction,parks_recreation,housing_commdevt
0,"Anchorage, AK",260900,259348,2000,0.005841,0.043473,5523.35,352.61,2426.51,2386.40,0.0,2386.40,195.60,0.0,0.0,0.0,0.0,0.0,213.45,0.0
1,"Anchorage, AK",263588,260816,2001,0.006632,0.043500,4831.29,594.42,2281.82,2251.80,0.0,2251.80,138.03,0.0,0.0,0.0,0.0,0.0,266.67,0.0
2,"Anchorage, AK",267280,264274,2002,0.006439,0.044706,4914.64,708.42,2455.38,2426.06,0.0,2426.06,165.42,0.0,0.0,0.0,0.0,0.0,285.35,0.0
3,"Anchorage, AK",282940,268229,2003,0.006789,0.045483,4463.29,320.32,2590.59,2561.61,0.0,2561.61,176.95,0.0,0.0,0.0,0.0,0.0,121.36,0.0
4,"Anchorage, AK",286671,270568,2004,0.008062,0.038187,5007.39,325.37,2627.83,2599.76,0.0,2599.76,143.11,0.0,0.0,0.0,0.0,0.0,136.75,0.0


### Load the Income Data

In [114]:
income_data= pd.read_csv('CleanData/DPI_Clean.csv')
povertydf = pd.read_csv('CleanData/Demographics.csv')
racedf = pd.read_csv('CleanData/Race.csv')
racedf = racedf[racedf['geographic_area_name'].notnull()]


income_data.loc[income_data.loc[income_data['city'].str.contains('Fort Smith, AR')].index.values,'city'] = 'Ft. Smith, AR'
income_data.loc[income_data.loc[income_data['city'].str.contains('Boise City, ID')].index.values,'city'] = 'Boise, ID'
income_data.loc[income_data.loc[income_data['city'].str.contains('Louisville/Jefferson County, KY')].index.values,'city'] = 'Louisville, KY'

povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Fort Smith, AR')].index.values,'city_name'] = 'Ft. Smith, AR'
povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Boise City, ID')].index.values,'city_name'] = 'Boise, ID'
povertydf.loc[povertydf.loc[povertydf['city_name'].str.contains('Louisville/Jefferson County, KY')].index.values,'city_name'] = 'Louisville, KY'

racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Fort Smith, AR')].index.values,'geographic_area_name'] = 'Ft. Smith, AR'
racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Boise City, ID')].index.values,'geographic_area_name'] = 'Boise, ID'
racedf.loc[racedf.loc[racedf['geographic_area_name'].str.contains('Louisville/Jefferson County, KY')].index.values,'geographic_area_name'] = 'Louisville, KY'

In [115]:
merged_df = budgets_fbi.merge(income_data,left_on=['city_merge_name','year'],right_on = ['city','Year'],how='inner')
merged_df.city.unique().shape

(119,)

### Lost Cities

In [116]:
for i in budgets_fbi.city_merge_name.unique():
    if i not in merged_df.city.unique():
        print(i)

Oakland, CA
Wilmington, DE
Ft. Lauderdale, FL
St. Petersburg, FL
Ft. Wayne, IN
Gary, IN
Kansas City, KS
St. Paul, MN
Ft. Worth, TX
Norfolk, VA
Tacoma, WA


In [117]:
merged_df2 = merged_df.merge(povertydf,left_on=['city_merge_name','year'],right_on = ['city_name','Year'],how='inner')
merged_df2.city.unique().shape

(118,)

In [118]:
for i in merged_df.city_merge_name.unique():
    if i not in merged_df2.city.unique():
        print(i)

Boise, ID


In [119]:
merged_df3 = merged_df2.merge(racedf,left_on=['city_merge_name','year'],right_on = ['geographic_area_name','Year'],how='inner')
merged_df3.city.unique().shape

(118,)

In [120]:
for i in merged_df.city_merge_name.unique():
    if i not in merged_df3.city.unique():
        print(i)

Boise, ID


In [129]:
merged_df[['city','Year']][merged_df['city_merge_name'].str.contains('ID')]

,city,Year
599,"Boise, ID",2000
600,"Boise, ID",2001
601,"Boise, ID",2002
